In [100]:
import pandas as pd
import json

import numpy as np
import matplotlib.pyplot as plt

import pickle 
import re

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
sns.set_style("darkgrid")
from copy import deepcopy
%pylab inline

%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


/Users/sdutta/anaconda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['extract']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [101]:
import datetime

In [102]:
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

In [103]:
with open('../Data/Finalnew_movies.json') as data_file:    
    data = json.load(data_file)

In [104]:
movie = pd.DataFrame(data)

In [105]:
movie

,Adj_Dom_Gross,Director,Director1,Domestic Gross,Domestic Gross1,F_Series,List1,Opening Weekend1,Release Date,Series Rank,Title,Title2,Widest Release1,Widest Release:,budget,genre,rating,run_time,url
0,"$17,251,200",None,Camille Delamarre,"$16,029,670","$16,029,670",Series: Transporter,"[Series: Transporter, Widest Opening Independe...",,"[September 4, 2015]","[4, 9]",The Transporter Refueled,"[The Transporter Refueled, $16,029,670, 4]",=,"3,434 theaters",N/A,Action,PG-13,1 hrs. 36 min.,/movies/?id=transporter4.htm
1,"$38,704,600",None,Luc Besson,"$25,296,447","$25,296,447",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",,"[October 11, 2002]","[3, 6]",The Transporter,"[The Transporter, $25,296,447, 3]",=,"2,610 theaters",$21 million,Action,PG-13,1 hrs. 32 min.,/movies/?id=transporter.htm
2,"$39,268,400",None,Olivier Megaton,"$31,715,062","$31,715,062",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",,"[November 26, 2008]","[2, 4]",Transporter 3,"[Transporter 3, $31,715,062, 2]",=,"2,626 theaters",N/A,Action,PG-13,1 hrs. 40 min.,/movies/?id=transporter3.htm
3,"$59,769,400",None,Louis Leterrier,"$43,095,856","$43,095,856",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",,"[September 2, 2005]","[1, 1, 3, 6]",Transporter 2,"[Transporter 2, $43,095,856, 1]",=,"3,320 theaters",$32 million,Action,PG-13,1 hrs. 28 min.,/movies/?id=transporter2.htm
4,"$2,264,100",None,Michael Winterbottom,"$2,030,962","$2,030,962",Series: The Trip,[Series: The Trip],,"[June 10, 2011]",[2],The Trip (2011),"[The Trip (2011), $2,030,962, 2]",=,62 theaters,N/A,Comedy,Unrated,1 hrs. 47 min.,/movies/?id=trip2011.htm
5,"$3,151,300",None,Michael Winterbottom,"$2,880,537","$2,880,537",Series: The Trip,[Series: The Trip],,"[August 15, 2014]",[1],The Trip to Italy,"[The Trip to Italy, $2,880,537, 1]",=,199 theaters,N/A,Comedy,Unrated,N/A,/movies/?id=triptoitaly.htm
6,"$155,432,600",None,Mike Mitchell,"$153,707,064","$153,707,064",Toy Adaptation,"[Toy Adaptation, Series: Trolls, Showdown: Tro...",,"[November 4, 2016]","[7, 1, 1]",Trolls,"[Trolls, $153,707,064, 7]",=,"4,066 theaters",$125 million,Animation,PG,1 hrs. 34 min.,/movies/?id=trolls.htm
7,"$237,246,700",None,Catherine Hardwicke,"$192,769,854","$192,769,854",Romantic Fantasy,"[Romantic Fantasy, Teen Romance, Vampire, Seri...",,"[November 21, 2008]","[8, 5, 5, 5, 5, 1, 1, 7, 4, 5, 9, 7, 7, 4]",Twilight,"[Twilight, $192,769,854, 8]",=,"3,649 theaters",$37 million,Romance,PG-13,2 hrs. 0 min.,/movies/?id=twilight08.htm
8,"$319,266,000",None,Bill Condon,"$281,287,133",Breaking Dawn Part 1,Off-Screen Couples On-Screen,"[Off-Screen Couples On-Screen, Pregnancy, Roma...",,"[November 18, 2011]","[3, 1, 5, 4, 4, 4, 4, 4, 6, 10, 10, 10, 8, 4, ...",The Twilight Saga:,"[The Twilight Saga:, Breaking Dawn Part 1, $28...",=,"4,066 theaters",$110 million,Romance,PG-13,1 hrs. 48 min.,/movies/?id=breakingdawn.htm
9,"$322,960,400",None,Bill Condon,"$292,324,737",Breaking Dawn Part 2,Romantic Fantasy,"[Romantic Fantasy, Teen Romance, Vampire, Were...",,"[November 16, 2012]","[4, 3, 3, 3, 10, 3, 3, 3, 5, 4, 6]",The Twilight Saga:,"[The Twilight Saga:, Breaking Dawn Part 2, $29...",=,"4,070 theaters",$120 million,Romance,PG-13,1 hrs. 56 min.,/movies/?id=breakingdawn2.htm


In [106]:
movies_df = movie

In [107]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784 entries, 0 to 783
Data columns (total 19 columns):
Adj_Dom_Gross       784 non-null object
Director            0 non-null object
Director1           784 non-null object
Domestic Gross      784 non-null object
Domestic Gross1     784 non-null object
F_Series            784 non-null object
List1               784 non-null object
Opening Weekend1    784 non-null object
Release Date        784 non-null object
Series Rank         784 non-null object
Title               784 non-null object
Title2              784 non-null object
Widest Release1     784 non-null object
Widest Release:     778 non-null object
budget              784 non-null object
genre               784 non-null object
rating              784 non-null object
run_time            784 non-null object
url                 784 non-null object
dtypes: object(19)
memory usage: 116.5+ KB


In [108]:
#Deleting Unwanted /Empty Columns
del movies_df['Director']
del movies_df['url']
del movies_df['Widest Release1']
del movies_df['Series Rank']
del movies_df['Opening Weekend1']

In [109]:
movies_df.head()

,Adj_Dom_Gross,Director1,Domestic Gross,Domestic Gross1,F_Series,List1,Release Date,Title,Title2,Widest Release:,budget,genre,rating,run_time
0,"$17,251,200",Camille Delamarre,"$16,029,670","$16,029,670",Series: Transporter,"[Series: Transporter, Widest Opening Independe...","[September 4, 2015]",The Transporter Refueled,"[The Transporter Refueled, $16,029,670, 4]","3,434 theaters",N/A,Action,PG-13,1 hrs. 36 min.
1,"$38,704,600",Luc Besson,"$25,296,447","$25,296,447",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...","[October 11, 2002]",The Transporter,"[The Transporter, $25,296,447, 3]","2,610 theaters",$21 million,Action,PG-13,1 hrs. 32 min.
2,"$39,268,400",Olivier Megaton,"$31,715,062","$31,715,062",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...","[November 26, 2008]",Transporter 3,"[Transporter 3, $31,715,062, 2]","2,626 theaters",N/A,Action,PG-13,1 hrs. 40 min.
3,"$59,769,400",Louis Leterrier,"$43,095,856","$43,095,856",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...","[September 2, 2005]",Transporter 2,"[Transporter 2, $43,095,856, 1]","3,320 theaters",$32 million,Action,PG-13,1 hrs. 28 min.
4,"$2,264,100",Michael Winterbottom,"$2,030,962","$2,030,962",Series: The Trip,[Series: The Trip],"[June 10, 2011]",The Trip (2011),"[The Trip (2011), $2,030,962, 2]",62 theaters,N/A,Comedy,Unrated,1 hrs. 47 min.


In [110]:
#Stripping movie Title 3 from Title2
def TitleStrip(x):
    if x[1][0] != '$':
        Title = x[0]+' '+x[1] 
    else:
        Title = x[0]
    return Title

In [111]:
movies_df['Title3'] = movies_df['Title2'].map(TitleStrip)

In [112]:
movies_df

,Adj_Dom_Gross,Director1,Domestic Gross,Domestic Gross1,F_Series,List1,Release Date,Title,Title2,Widest Release:,budget,genre,rating,run_time,Title3
0,"$17,251,200",Camille Delamarre,"$16,029,670","$16,029,670",Series: Transporter,"[Series: Transporter, Widest Opening Independe...","[September 4, 2015]",The Transporter Refueled,"[The Transporter Refueled, $16,029,670, 4]","3,434 theaters",N/A,Action,PG-13,1 hrs. 36 min.,The Transporter Refueled
1,"$38,704,600",Luc Besson,"$25,296,447","$25,296,447",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...","[October 11, 2002]",The Transporter,"[The Transporter, $25,296,447, 3]","2,610 theaters",$21 million,Action,PG-13,1 hrs. 32 min.,The Transporter
2,"$39,268,400",Olivier Megaton,"$31,715,062","$31,715,062",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...","[November 26, 2008]",Transporter 3,"[Transporter 3, $31,715,062, 2]","2,626 theaters",N/A,Action,PG-13,1 hrs. 40 min.,Transporter 3
3,"$59,769,400",Louis Leterrier,"$43,095,856","$43,095,856",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...","[September 2, 2005]",Transporter 2,"[Transporter 2, $43,095,856, 1]","3,320 theaters",$32 million,Action,PG-13,1 hrs. 28 min.,Transporter 2
4,"$2,264,100",Michael Winterbottom,"$2,030,962","$2,030,962",Series: The Trip,[Series: The Trip],"[June 10, 2011]",The Trip (2011),"[The Trip (2011), $2,030,962, 2]",62 theaters,N/A,Comedy,Unrated,1 hrs. 47 min.,The Trip (2011)
5,"$3,151,300",Michael Winterbottom,"$2,880,537","$2,880,537",Series: The Trip,[Series: The Trip],"[August 15, 2014]",The Trip to Italy,"[The Trip to Italy, $2,880,537, 1]",199 theaters,N/A,Comedy,Unrated,N/A,The Trip to Italy
6,"$155,432,600",Mike Mitchell,"$153,707,064","$153,707,064",Toy Adaptation,"[Toy Adaptation, Series: Trolls, Showdown: Tro...","[November 4, 2016]",Trolls,"[Trolls, $153,707,064, 7]","4,066 theaters",$125 million,Animation,PG,1 hrs. 34 min.,Trolls
7,"$237,246,700",Catherine Hardwicke,"$192,769,854","$192,769,854",Romantic Fantasy,"[Romantic Fantasy, Teen Romance, Vampire, Seri...","[November 21, 2008]",Twilight,"[Twilight, $192,769,854, 8]","3,649 theaters",$37 million,Romance,PG-13,2 hrs. 0 min.,Twilight
8,"$319,266,000",Bill Condon,"$281,287,133",Breaking Dawn Part 1,Off-Screen Couples On-Screen,"[Off-Screen Couples On-Screen, Pregnancy, Roma...","[November 18, 2011]",The Twilight Saga:,"[The Twilight Saga:, Breaking Dawn Part 1, $28...","4,066 theaters",$110 million,Romance,PG-13,1 hrs. 48 min.,The Twilight Saga: Breaking Dawn Part 1
9,"$322,960,400",Bill Condon,"$292,324,737",Breaking Dawn Part 2,Romantic Fantasy,"[Romantic Fantasy, Teen Romance, Vampire, Were...","[November 16, 2012]",The Twilight Saga:,"[The Twilight Saga:, Breaking Dawn Part 2, $29...","4,070 theaters",$120 million,Romance,PG-13,1 hrs. 56 min.,The Twilight Saga: Breaking Dawn Part 2


In [113]:
# Changing dates from list entries to strings
def bracs(x):
    # that, if x is a string,
    if x != []:
        d = str(x[0])
        a = d.replace(',', ' ')
        return a
    else:
        return None


movies_df['Release Date']= movies_df['Release Date'].map(bracs)

In [114]:
# Change to Datetime 
movies_df['My_Rel_Date'] = pd.to_datetime(movies_df['Release Date'])
movies_df.head()

,Adj_Dom_Gross,Director1,Domestic Gross,Domestic Gross1,F_Series,List1,Release Date,Title,Title2,Widest Release:,budget,genre,rating,run_time,Title3,My_Rel_Date
0,"$17,251,200",Camille Delamarre,"$16,029,670","$16,029,670",Series: Transporter,"[Series: Transporter, Widest Opening Independe...",September 4 2015,The Transporter Refueled,"[The Transporter Refueled, $16,029,670, 4]","3,434 theaters",N/A,Action,PG-13,1 hrs. 36 min.,The Transporter Refueled,2015-09-04
1,"$38,704,600",Luc Besson,"$25,296,447","$25,296,447",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",October 11 2002,The Transporter,"[The Transporter, $25,296,447, 3]","2,610 theaters",$21 million,Action,PG-13,1 hrs. 32 min.,The Transporter,2002-10-11
2,"$39,268,400",Olivier Megaton,"$31,715,062","$31,715,062",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",November 26 2008,Transporter 3,"[Transporter 3, $31,715,062, 2]","2,626 theaters",N/A,Action,PG-13,1 hrs. 40 min.,Transporter 3,2008-11-26
3,"$59,769,400",Louis Leterrier,"$43,095,856","$43,095,856",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",September 2 2005,Transporter 2,"[Transporter 2, $43,095,856, 1]","3,320 theaters",$32 million,Action,PG-13,1 hrs. 28 min.,Transporter 2,2005-09-02
4,"$2,264,100",Michael Winterbottom,"$2,030,962","$2,030,962",Series: The Trip,[Series: The Trip],June 10 2011,The Trip (2011),"[The Trip (2011), $2,030,962, 2]",62 theaters,N/A,Comedy,Unrated,1 hrs. 47 min.,The Trip (2011),2011-06-10


In [115]:
# Making new column of Month
movies_df['Month'] = movies_df['My_Rel_Date'].dt.month

In [116]:
movies_df.head()

,Adj_Dom_Gross,Director1,Domestic Gross,Domestic Gross1,F_Series,List1,Release Date,Title,Title2,Widest Release:,budget,genre,rating,run_time,Title3,My_Rel_Date,Month
0,"$17,251,200",Camille Delamarre,"$16,029,670","$16,029,670",Series: Transporter,"[Series: Transporter, Widest Opening Independe...",September 4 2015,The Transporter Refueled,"[The Transporter Refueled, $16,029,670, 4]","3,434 theaters",N/A,Action,PG-13,1 hrs. 36 min.,The Transporter Refueled,2015-09-04,9.0
1,"$38,704,600",Luc Besson,"$25,296,447","$25,296,447",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",October 11 2002,The Transporter,"[The Transporter, $25,296,447, 3]","2,610 theaters",$21 million,Action,PG-13,1 hrs. 32 min.,The Transporter,2002-10-11,10.0
2,"$39,268,400",Olivier Megaton,"$31,715,062","$31,715,062",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",November 26 2008,Transporter 3,"[Transporter 3, $31,715,062, 2]","2,626 theaters",N/A,Action,PG-13,1 hrs. 40 min.,Transporter 3,2008-11-26,11.0
3,"$59,769,400",Louis Leterrier,"$43,095,856","$43,095,856",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",September 2 2005,Transporter 2,"[Transporter 2, $43,095,856, 1]","3,320 theaters",$32 million,Action,PG-13,1 hrs. 28 min.,Transporter 2,2005-09-02,9.0
4,"$2,264,100",Michael Winterbottom,"$2,030,962","$2,030,962",Series: The Trip,[Series: The Trip],June 10 2011,The Trip (2011),"[The Trip (2011), $2,030,962, 2]",62 theaters,N/A,Comedy,Unrated,1 hrs. 47 min.,The Trip (2011),2011-06-10,6.0


In [117]:
#Defining Season

def Season(x):
    RelSeason =""
    if (x == 4.0 or x==5.0):
        RelSeason = "Spring"
    if (x == 6.0 or x==7.0 or x== 8.0):
        RelSeason = "Summer"
    if (x == 9.0 or x==10.0 or x== 11.0):
        RelSeason = "Fall"
    if (x == 12.0 or x==1.0 or x== 2.0 or x== 3.0 ):
        RelSeason = "Winter"
    return RelSeason 

movies_df['Season'] = movies_df['Month'].map(Season)
movies_df.head()

,Adj_Dom_Gross,Director1,Domestic Gross,Domestic Gross1,F_Series,List1,Release Date,Title,Title2,Widest Release:,budget,genre,rating,run_time,Title3,My_Rel_Date,Month,Season
0,"$17,251,200",Camille Delamarre,"$16,029,670","$16,029,670",Series: Transporter,"[Series: Transporter, Widest Opening Independe...",September 4 2015,The Transporter Refueled,"[The Transporter Refueled, $16,029,670, 4]","3,434 theaters",N/A,Action,PG-13,1 hrs. 36 min.,The Transporter Refueled,2015-09-04,9.0,Fall
1,"$38,704,600",Luc Besson,"$25,296,447","$25,296,447",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",October 11 2002,The Transporter,"[The Transporter, $25,296,447, 3]","2,610 theaters",$21 million,Action,PG-13,1 hrs. 32 min.,The Transporter,2002-10-11,10.0,Fall
2,"$39,268,400",Olivier Megaton,"$31,715,062","$31,715,062",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",November 26 2008,Transporter 3,"[Transporter 3, $31,715,062, 2]","2,626 theaters",N/A,Action,PG-13,1 hrs. 40 min.,Transporter 3,2008-11-26,11.0,Fall
3,"$59,769,400",Louis Leterrier,"$43,095,856","$43,095,856",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",September 2 2005,Transporter 2,"[Transporter 2, $43,095,856, 1]","3,320 theaters",$32 million,Action,PG-13,1 hrs. 28 min.,Transporter 2,2005-09-02,9.0,Fall
4,"$2,264,100",Michael Winterbottom,"$2,030,962","$2,030,962",Series: The Trip,[Series: The Trip],June 10 2011,The Trip (2011),"[The Trip (2011), $2,030,962, 2]",62 theaters,N/A,Comedy,Unrated,1 hrs. 47 min.,The Trip (2011),2011-06-10,6.0,Summer


In [118]:
# Cleaning up Widest Release
movies_df['Widest Release:'] = movies_df['Widest Release:'].str.replace(',', '')
movies_df['Widest Release:'] = movies_df['Widest Release:'].str.replace('[a-z," "]+', '')
movies_df

,Adj_Dom_Gross,Director1,Domestic Gross,Domestic Gross1,F_Series,List1,Release Date,Title,Title2,Widest Release:,budget,genre,rating,run_time,Title3,My_Rel_Date,Month,Season
0,"$17,251,200",Camille Delamarre,"$16,029,670","$16,029,670",Series: Transporter,"[Series: Transporter, Widest Opening Independe...",September 4 2015,The Transporter Refueled,"[The Transporter Refueled, $16,029,670, 4]",3434,N/A,Action,PG-13,1 hrs. 36 min.,The Transporter Refueled,2015-09-04,9.0,Fall
1,"$38,704,600",Luc Besson,"$25,296,447","$25,296,447",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",October 11 2002,The Transporter,"[The Transporter, $25,296,447, 3]",2610,$21 million,Action,PG-13,1 hrs. 32 min.,The Transporter,2002-10-11,10.0,Fall
2,"$39,268,400",Olivier Megaton,"$31,715,062","$31,715,062",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",November 26 2008,Transporter 3,"[Transporter 3, $31,715,062, 2]",2626,N/A,Action,PG-13,1 hrs. 40 min.,Transporter 3,2008-11-26,11.0,Fall
3,"$59,769,400",Louis Leterrier,"$43,095,856","$43,095,856",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",September 2 2005,Transporter 2,"[Transporter 2, $43,095,856, 1]",3320,$32 million,Action,PG-13,1 hrs. 28 min.,Transporter 2,2005-09-02,9.0,Fall
4,"$2,264,100",Michael Winterbottom,"$2,030,962","$2,030,962",Series: The Trip,[Series: The Trip],June 10 2011,The Trip (2011),"[The Trip (2011), $2,030,962, 2]",62,N/A,Comedy,Unrated,1 hrs. 47 min.,The Trip (2011),2011-06-10,6.0,Summer
5,"$3,151,300",Michael Winterbottom,"$2,880,537","$2,880,537",Series: The Trip,[Series: The Trip],August 15 2014,The Trip to Italy,"[The Trip to Italy, $2,880,537, 1]",199,N/A,Comedy,Unrated,N/A,The Trip to Italy,2014-08-15,8.0,Summer
6,"$155,432,600",Mike Mitchell,"$153,707,064","$153,707,064",Toy Adaptation,"[Toy Adaptation, Series: Trolls, Showdown: Tro...",November 4 2016,Trolls,"[Trolls, $153,707,064, 7]",4066,$125 million,Animation,PG,1 hrs. 34 min.,Trolls,2016-11-04,11.0,Fall
7,"$237,246,700",Catherine Hardwicke,"$192,769,854","$192,769,854",Romantic Fantasy,"[Romantic Fantasy, Teen Romance, Vampire, Seri...",November 21 2008,Twilight,"[Twilight, $192,769,854, 8]",3649,$37 million,Romance,PG-13,2 hrs. 0 min.,Twilight,2008-11-21,11.0,Fall
8,"$319,266,000",Bill Condon,"$281,287,133",Breaking Dawn Part 1,Off-Screen Couples On-Screen,"[Off-Screen Couples On-Screen, Pregnancy, Roma...",November 18 2011,The Twilight Saga:,"[The Twilight Saga:, Breaking Dawn Part 1, $28...",4066,$110 million,Romance,PG-13,1 hrs. 48 min.,The Twilight Saga: Breaking Dawn Part 1,2011-11-18,11.0,Fall
9,"$322,960,400",Bill Condon,"$292,324,737",Breaking Dawn Part 2,Romantic Fantasy,"[Romantic Fantasy, Teen Romance, Vampire, Were...",November 16 2012,The Twilight Saga:,"[The Twilight Saga:, Breaking Dawn Part 2, $29...",4070,$120 million,Romance,PG-13,1 hrs. 56 min.,The Twilight Saga: Breaking Dawn Part 2,2012-11-16,11.0,Fall


In [119]:
# Widest Release :  Extracting number of theaters from unicode values and changing them into float

def extract(x):
    out = re.findall(r"\d+",str(x))
    if len(out)>0:
        return int(out[0])
    else:
        return np.nan


In [120]:
movies_df['Widest Release:'] = movies_df['Widest Release:'].map(extract)

In [121]:
movies_df['Widest Release:'].dtypes
movies_df.dtypes

Adj_Dom_Gross              object
Director1                  object
Domestic Gross             object
Domestic Gross1            object
F_Series                   object
List1                      object
Release Date               object
Title                      object
Title2                     object
Widest Release:           float64
budget                     object
genre                      object
rating                     object
run_time                   object
Title3                     object
My_Rel_Date        datetime64[ns]
Month                     float64
Season                     object
dtype: object

In [122]:
# Cleaning up List1 to get Franchise Name
def FindSeries(x):
    series =''
    for i in x:
        if 'Series:' in i:
            series = ''.join(i.split()[1:])
            return series

movies_df['Franchise'] = movies_df['List1'].map(FindSeries)
movies_df

,Adj_Dom_Gross,Director1,Domestic Gross,Domestic Gross1,F_Series,List1,Release Date,Title,Title2,Widest Release:,budget,genre,rating,run_time,Title3,My_Rel_Date,Month,Season,Franchise
0,"$17,251,200",Camille Delamarre,"$16,029,670","$16,029,670",Series: Transporter,"[Series: Transporter, Widest Opening Independe...",September 4 2015,The Transporter Refueled,"[The Transporter Refueled, $16,029,670, 4]",3434.0,N/A,Action,PG-13,1 hrs. 36 min.,The Transporter Refueled,2015-09-04,9.0,Fall,Transporter
1,"$38,704,600",Luc Besson,"$25,296,447","$25,296,447",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",October 11 2002,The Transporter,"[The Transporter, $25,296,447, 3]",2610.0,$21 million,Action,PG-13,1 hrs. 32 min.,The Transporter,2002-10-11,10.0,Fall,Transporter
2,"$39,268,400",Olivier Megaton,"$31,715,062","$31,715,062",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",November 26 2008,Transporter 3,"[Transporter 3, $31,715,062, 2]",2626.0,N/A,Action,PG-13,1 hrs. 40 min.,Transporter 3,2008-11-26,11.0,Fall,Transporter
3,"$59,769,400",Louis Leterrier,"$43,095,856","$43,095,856",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",September 2 2005,Transporter 2,"[Transporter 2, $43,095,856, 1]",3320.0,$32 million,Action,PG-13,1 hrs. 28 min.,Transporter 2,2005-09-02,9.0,Fall,Transporter
4,"$2,264,100",Michael Winterbottom,"$2,030,962","$2,030,962",Series: The Trip,[Series: The Trip],June 10 2011,The Trip (2011),"[The Trip (2011), $2,030,962, 2]",62.0,N/A,Comedy,Unrated,1 hrs. 47 min.,The Trip (2011),2011-06-10,6.0,Summer,TheTrip
5,"$3,151,300",Michael Winterbottom,"$2,880,537","$2,880,537",Series: The Trip,[Series: The Trip],August 15 2014,The Trip to Italy,"[The Trip to Italy, $2,880,537, 1]",199.0,N/A,Comedy,Unrated,N/A,The Trip to Italy,2014-08-15,8.0,Summer,TheTrip
6,"$155,432,600",Mike Mitchell,"$153,707,064","$153,707,064",Toy Adaptation,"[Toy Adaptation, Series: Trolls, Showdown: Tro...",November 4 2016,Trolls,"[Trolls, $153,707,064, 7]",4066.0,$125 million,Animation,PG,1 hrs. 34 min.,Trolls,2016-11-04,11.0,Fall,Trolls
7,"$237,246,700",Catherine Hardwicke,"$192,769,854","$192,769,854",Romantic Fantasy,"[Romantic Fantasy, Teen Romance, Vampire, Seri...",November 21 2008,Twilight,"[Twilight, $192,769,854, 8]",3649.0,$37 million,Romance,PG-13,2 hrs. 0 min.,Twilight,2008-11-21,11.0,Fall,Twilight
8,"$319,266,000",Bill Condon,"$281,287,133",Breaking Dawn Part 1,Off-Screen Couples On-Screen,"[Off-Screen Couples On-Screen, Pregnancy, Roma...",November 18 2011,The Twilight Saga:,"[The Twilight Saga:, Breaking Dawn Part 1, $28...",4066.0,$110 million,Romance,PG-13,1 hrs. 48 min.,The Twilight Saga: Breaking Dawn Part 1,2011-11-18,11.0,Fall,Twilight
9,"$322,960,400",Bill Condon,"$292,324,737",Breaking Dawn Part 2,Romantic Fantasy,"[Romantic Fantasy, Teen Romance, Vampire, Were...",November 16 2012,The Twilight Saga:,"[The Twilight Saga:, Breaking Dawn Part 2, $29...",4070.0,$120 million,Romance,PG-13,1 hrs. 56 min.,The Twilight Saga: Breaking Dawn Part 2,2012-11-16,11.0,Fall,Twilight


In [123]:

# Converting RunTime to minutes
def Time(x):
    if x!= 'N/A':
        tim = []
        tim.extend(x.split())
        time = int(tim[0])*60 + int(tim[2])
        return time
    else:
        return np.nan

movies_df['Run_Time'] = movies_df['run_time'].map(Time)

In [124]:
#Clean up Budget

def StripBudget(x):
    if x != 'N/A':
        a=x.replace(',', '')
        a=a.replace('$', '')
        if 'million' in a:
            a = a.replace('million', '')
            a = a.strip()
            a = float(a)*1000000
        return float(a)
    else:
        return np.nan

In [125]:
movies_df['budget'] = movies_df['budget'].map(StripBudget)

In [126]:
movies_df

,Adj_Dom_Gross,Director1,Domestic Gross,Domestic Gross1,F_Series,List1,Release Date,Title,Title2,Widest Release:,budget,genre,rating,run_time,Title3,My_Rel_Date,Month,Season,Franchise,Run_Time
0,"$17,251,200",Camille Delamarre,"$16,029,670","$16,029,670",Series: Transporter,"[Series: Transporter, Widest Opening Independe...",September 4 2015,The Transporter Refueled,"[The Transporter Refueled, $16,029,670, 4]",3434.0,NaN,Action,PG-13,1 hrs. 36 min.,The Transporter Refueled,2015-09-04,9.0,Fall,Transporter,96.0
1,"$38,704,600",Luc Besson,"$25,296,447","$25,296,447",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",October 11 2002,The Transporter,"[The Transporter, $25,296,447, 3]",2610.0,2.100e+07,Action,PG-13,1 hrs. 32 min.,The Transporter,2002-10-11,10.0,Fall,Transporter,92.0
2,"$39,268,400",Olivier Megaton,"$31,715,062","$31,715,062",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",November 26 2008,Transporter 3,"[Transporter 3, $31,715,062, 2]",2626.0,NaN,Action,PG-13,1 hrs. 40 min.,Transporter 3,2008-11-26,11.0,Fall,Transporter,100.0
3,"$59,769,400",Louis Leterrier,"$43,095,856","$43,095,856",Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",September 2 2005,Transporter 2,"[Transporter 2, $43,095,856, 1]",3320.0,3.200e+07,Action,PG-13,1 hrs. 28 min.,Transporter 2,2005-09-02,9.0,Fall,Transporter,88.0
4,"$2,264,100",Michael Winterbottom,"$2,030,962","$2,030,962",Series: The Trip,[Series: The Trip],June 10 2011,The Trip (2011),"[The Trip (2011), $2,030,962, 2]",62.0,NaN,Comedy,Unrated,1 hrs. 47 min.,The Trip (2011),2011-06-10,6.0,Summer,TheTrip,107.0
5,"$3,151,300",Michael Winterbottom,"$2,880,537","$2,880,537",Series: The Trip,[Series: The Trip],August 15 2014,The Trip to Italy,"[The Trip to Italy, $2,880,537, 1]",199.0,NaN,Comedy,Unrated,N/A,The Trip to Italy,2014-08-15,8.0,Summer,TheTrip,NaN
6,"$155,432,600",Mike Mitchell,"$153,707,064","$153,707,064",Toy Adaptation,"[Toy Adaptation, Series: Trolls, Showdown: Tro...",November 4 2016,Trolls,"[Trolls, $153,707,064, 7]",4066.0,1.250e+08,Animation,PG,1 hrs. 34 min.,Trolls,2016-11-04,11.0,Fall,Trolls,94.0
7,"$237,246,700",Catherine Hardwicke,"$192,769,854","$192,769,854",Romantic Fantasy,"[Romantic Fantasy, Teen Romance, Vampire, Seri...",November 21 2008,Twilight,"[Twilight, $192,769,854, 8]",3649.0,3.700e+07,Romance,PG-13,2 hrs. 0 min.,Twilight,2008-11-21,11.0,Fall,Twilight,120.0
8,"$319,266,000",Bill Condon,"$281,287,133",Breaking Dawn Part 1,Off-Screen Couples On-Screen,"[Off-Screen Couples On-Screen, Pregnancy, Roma...",November 18 2011,The Twilight Saga:,"[The Twilight Saga:, Breaking Dawn Part 1, $28...",4066.0,1.100e+08,Romance,PG-13,1 hrs. 48 min.,The Twilight Saga: Breaking Dawn Part 1,2011-11-18,11.0,Fall,Twilight,108.0
9,"$322,960,400",Bill Condon,"$292,324,737",Breaking Dawn Part 2,Romantic Fantasy,"[Romantic Fantasy, Teen Romance, Vampire, Were...",November 16 2012,The Twilight Saga:,"[The Twilight Saga:, Breaking Dawn Part 2, $29...",4070.0,1.200e+08,Romance,PG-13,1 hrs. 56 min.,The Twilight Saga: Breaking Dawn Part 2,2012-11-16,11.0,Fall,Twilight,116.0


In [127]:
#Striping $ sign and commas

def StripMoney(x):
    if x != np.nan:
        a=x.replace(',', '')
        a=a.replace('$', '')
        return int(a)
    else:
        return None

In [128]:
# Adj_Dom_Gross -Strip $ and covert to float data type

movies_df['Adj_Dom_Gross'] = movies_df['Adj_Dom_Gross'].map(StripMoney)
movies_df['Adj_Dom_Gross'] = movies_df['Adj_Dom_Gross'].astype(int)
movies_df['Adj_Dom_Gross'].dtypes

dtype('int64')

In [129]:
# Adj_Dom_Gross -Strip $ and covert to float data type

movies_df['Domestic Gross'] = movies_df['Domestic Gross'].map(StripMoney)
movies_df['Domestic Gross'] = movies_df['Domestic Gross'].astype(int)
movies_df['Domestic Gross'].dtypes

dtype('int64')

In [130]:
del movies_df['Domestic Gross1']


In [131]:
movies_df.rename(columns={'Title3': 'MovieTitle','Widest Release:': 'Widest Release', 'Director1':'Director','My_Rel_Date':'Release_Date' }, inplace=True)

In [132]:
movies_df

,Adj_Dom_Gross,Director,Domestic Gross,F_Series,List1,Release Date,Title,Title2,Widest Release,budget,genre,rating,run_time,MovieTitle,Release_Date,Month,Season,Franchise,Run_Time
0,17251200,Camille Delamarre,16029670,Series: Transporter,"[Series: Transporter, Widest Opening Independe...",September 4 2015,The Transporter Refueled,"[The Transporter Refueled, $16,029,670, 4]",3434.0,NaN,Action,PG-13,1 hrs. 36 min.,The Transporter Refueled,2015-09-04,9.0,Fall,Transporter,96.0
1,38704600,Luc Besson,25296447,Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",October 11 2002,The Transporter,"[The Transporter, $25,296,447, 3]",2610.0,2.100e+07,Action,PG-13,1 hrs. 32 min.,The Transporter,2002-10-11,10.0,Fall,Transporter,92.0
2,39268400,Olivier Megaton,31715062,Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",November 26 2008,Transporter 3,"[Transporter 3, $31,715,062, 2]",2626.0,NaN,Action,PG-13,1 hrs. 40 min.,Transporter 3,2008-11-26,11.0,Fall,Transporter,100.0
3,59769400,Louis Leterrier,43095856,Series: Transporter,"[Series: Transporter, Showdown: 'Transporter' ...",September 2 2005,Transporter 2,"[Transporter 2, $43,095,856, 1]",3320.0,3.200e+07,Action,PG-13,1 hrs. 28 min.,Transporter 2,2005-09-02,9.0,Fall,Transporter,88.0
4,2264100,Michael Winterbottom,2030962,Series: The Trip,[Series: The Trip],June 10 2011,The Trip (2011),"[The Trip (2011), $2,030,962, 2]",62.0,NaN,Comedy,Unrated,1 hrs. 47 min.,The Trip (2011),2011-06-10,6.0,Summer,TheTrip,107.0
5,3151300,Michael Winterbottom,2880537,Series: The Trip,[Series: The Trip],August 15 2014,The Trip to Italy,"[The Trip to Italy, $2,880,537, 1]",199.0,NaN,Comedy,Unrated,N/A,The Trip to Italy,2014-08-15,8.0,Summer,TheTrip,NaN
6,155432600,Mike Mitchell,153707064,Toy Adaptation,"[Toy Adaptation, Series: Trolls, Showdown: Tro...",November 4 2016,Trolls,"[Trolls, $153,707,064, 7]",4066.0,1.250e+08,Animation,PG,1 hrs. 34 min.,Trolls,2016-11-04,11.0,Fall,Trolls,94.0
7,237246700,Catherine Hardwicke,192769854,Romantic Fantasy,"[Romantic Fantasy, Teen Romance, Vampire, Seri...",November 21 2008,Twilight,"[Twilight, $192,769,854, 8]",3649.0,3.700e+07,Romance,PG-13,2 hrs. 0 min.,Twilight,2008-11-21,11.0,Fall,Twilight,120.0
8,319266000,Bill Condon,281287133,Off-Screen Couples On-Screen,"[Off-Screen Couples On-Screen, Pregnancy, Roma...",November 18 2011,The Twilight Saga:,"[The Twilight Saga:, Breaking Dawn Part 1, $28...",4066.0,1.100e+08,Romance,PG-13,1 hrs. 48 min.,The Twilight Saga: Breaking Dawn Part 1,2011-11-18,11.0,Fall,Twilight,108.0
9,322960400,Bill Condon,292324737,Romantic Fantasy,"[Romantic Fantasy, Teen Romance, Vampire, Were...",November 16 2012,The Twilight Saga:,"[The Twilight Saga:, Breaking Dawn Part 2, $29...",4070.0,1.200e+08,Romance,PG-13,1 hrs. 56 min.,The Twilight Saga: Breaking Dawn Part 2,2012-11-16,11.0,Fall,Twilight,116.0


In [133]:
Franchise_Movies= movies_df.filter(['MovieTitle','Franchise','Release_Date',\
                                    'budget','Domestic Gross','Adj_Dom_Gross','genre','Widest Release','Season','Month','rating','Run_Time','Director'])



In [134]:
Franchise_Movies

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director
0,The Transporter Refueled,Transporter,2015-09-04,NaN,16029670,17251200,Action,3434.0,Fall,9.0,PG-13,96.0,Camille Delamarre
1,The Transporter,Transporter,2002-10-11,2.100e+07,25296447,38704600,Action,2610.0,Fall,10.0,PG-13,92.0,Luc Besson
2,Transporter 3,Transporter,2008-11-26,NaN,31715062,39268400,Action,2626.0,Fall,11.0,PG-13,100.0,Olivier Megaton
3,Transporter 2,Transporter,2005-09-02,3.200e+07,43095856,59769400,Action,3320.0,Fall,9.0,PG-13,88.0,Louis Leterrier
4,The Trip (2011),TheTrip,2011-06-10,NaN,2030962,2264100,Comedy,62.0,Summer,6.0,Unrated,107.0,Michael Winterbottom
5,The Trip to Italy,TheTrip,2014-08-15,NaN,2880537,3151300,Comedy,199.0,Summer,8.0,Unrated,NaN,Michael Winterbottom
6,Trolls,Trolls,2016-11-04,1.250e+08,153707064,155432600,Animation,4066.0,Fall,11.0,PG,94.0,Mike Mitchell
7,Twilight,Twilight,2008-11-21,3.700e+07,192769854,237246700,Romance,3649.0,Fall,11.0,PG-13,120.0,Catherine Hardwicke
8,The Twilight Saga: Breaking Dawn Part 1,Twilight,2011-11-18,1.100e+08,281287133,319266000,Romance,4066.0,Fall,11.0,PG-13,108.0,Bill Condon
9,The Twilight Saga: Breaking Dawn Part 2,Twilight,2012-11-16,1.200e+08,292324737,322960400,Romance,4070.0,Fall,11.0,PG-13,116.0,Bill Condon


### Adding Franchise Sequence Col ---
Filling in misssing values  
Sorting by Franchise Name and Rel Date 

In [135]:
Franchise_Movies.loc[Franchise_Movies.MovieTitle == 'Logan', 'Franchise'] = 'X-Men'

In [136]:
len(Franchise_Movies[(Franchise_Movies['Franchise'].isnull())])

16

In [137]:
mask = (Franchise_Movies['Franchise'].isnull())
Franchise_Movies[(mask)]

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director
122,Attack of the Clones: The IMAX Experience (IMAX),None,2002-11-01,NaN,8485488,12945700,IMAX,58.0,Fall,11.0,PG,120.0,Brand: Lucasfilm
129,Star Trek V: The Final Frontier,None,1989-06-09,NaN,52210049,116913700,Sci-Fi Adventure,2202.0,Summer,6.0,PG,107.0,Leonard Nimoy
130,Star Trek: Insurrection,None,1998-12-11,5.800e+07,70187658,130671400,Sci-Fi Adventure,2677.0,Winter,12.0,PG,103.0,Patrick Stewart
141,Return of the Jedi (Re-issue),None,1985-03-29,NaN,11252123,28177900,Sci-Fi Fantasy,894.0,Winter,3.0,PG,134.0,Brand: Lucasfilm
142,The Empire Strikes Back (Re-issue),None,1982-11-19,NaN,13276241,40144800,Sci-Fi Fantasy,1006.0,Fall,11.0,PG,124.0,Brand: Lucasfilm
143,Star Wars (Re-issue),None,1982-08-13,NaN,15476285,46797300,Sci-Fi Fantasy,1070.0,Summer,8.0,PG,121.0,Brand: Lucasfilm
144,Star Wars: The Clone Wars,None,2008-08-15,NaN,35161554,43535700,Animation,3452.0,Summer,8.0,PG,98.0,George Lucas
145,Star Wars: Episode I - The Phantom Menace (in 3D),None,2012-02-10,NaN,43456382,48772900,Sci-Fi Fantasy,2655.0,Winter,2.0,PG,133.0,Brand: Lucasfilm
146,Return of the Jedi (Special Edition),None,1997-03-14,NaN,45470437,88068000,Sci-Fi Fantasy,2111.0,Winter,3.0,PG,135.0,Brand: Lucasfilm
467,The Incredible Hulk,None,2008-06-13,1.500e+08,134806913,166912700,Action / Adventure,3508.0,Summer,6.0,PG-13,114.0,Louis Leterrier


In [138]:
Franchise_Movies.loc[Franchise_Movies.MovieTitle == 'Star Trek V: The Final Frontier', 'Franchise'] = 'Star Trek'
Franchise_Movies.loc[Franchise_Movies.MovieTitle == 'Star Trek: Insurrection', 'Franchise'] = 'Star Trek'
Franchise_Movies.loc[Franchise_Movies.MovieTitle == 'Star Wars: The Clone Wars', 'Franchise'] = 'Star Wars'
Franchise_Movies.loc[Franchise_Movies.MovieTitle == 'The Incredible Hulk', 'Franchise'] = 'Marvel'
Franchise_Movies.loc[Franchise_Movies.MovieTitle == 'Ant-Man', 'Franchise'] = 'Marvel'
Franchise_Movies.loc[Franchise_Movies.MovieTitle == 'Doctor Strange', 'Franchise'] = 'Marvel'
Franchise_Movies.loc[Franchise_Movies.MovieTitle == 'Licence to Kill', 'Franchise'] = 'James Bond'
Franchise_Movies.loc[Franchise_Movies.MovieTitle == 'A View to a Kill', 'Franchise'] = 'James Bond'
Franchise_Movies.loc[Franchise_Movies.MovieTitle == 'The Living Daylights', 'Franchise'] = 'Star Wars'
Franchise_Movies.loc[Franchise_Movies.MovieTitle == 'Never Say Never Again', 'Franchise'] = 'James Bond'

In [139]:
mask = (Franchise_Movies['Franchise'].isnull())
Franchise_Movies[(mask)]

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director
122,Attack of the Clones: The IMAX Experience (IMAX),None,2002-11-01,NaN,8485488,12945700,IMAX,58.0,Fall,11.0,PG,120.0,Brand: Lucasfilm
141,Return of the Jedi (Re-issue),None,1985-03-29,NaN,11252123,28177900,Sci-Fi Fantasy,894.0,Winter,3.0,PG,134.0,Brand: Lucasfilm
142,The Empire Strikes Back (Re-issue),None,1982-11-19,NaN,13276241,40144800,Sci-Fi Fantasy,1006.0,Fall,11.0,PG,124.0,Brand: Lucasfilm
143,Star Wars (Re-issue),None,1982-08-13,NaN,15476285,46797300,Sci-Fi Fantasy,1070.0,Summer,8.0,PG,121.0,Brand: Lucasfilm
145,Star Wars: Episode I - The Phantom Menace (in 3D),None,2012-02-10,NaN,43456382,48772900,Sci-Fi Fantasy,2655.0,Winter,2.0,PG,133.0,Brand: Lucasfilm
146,Return of the Jedi (Special Edition),None,1997-03-14,NaN,45470437,88068000,Sci-Fi Fantasy,2111.0,Winter,3.0,PG,135.0,Brand: Lucasfilm


In [140]:
# Sorting by Franchise and Date
Franchise_Movies = Franchise_Movies.sort_values(by=['Franchise','Release_Date']).reset_index()
del Franchise_Movies['index']
Franchise_Movies.head()

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director
0,300,300,2007-03-09,6.500e+07,210614939,272146300,Period Action,3280.0,Winter,3.0,R,117.0,Zack Snyder
1,300: Rise of An Empire,300,2014-03-07,1.100e+08,106580051,118549400,Period Action,3490.0,Winter,3.0,R,102.0,Noam Murro
2,3 Ninjas,3Ninjas,1992-08-07,NaN,29000301,62123600,Family Adventure,1954.0,Summer,8.0,PG,85.0,Jon Turteltaub
3,3 Ninjas Kick Back,3Ninjas,1994-05-06,NaN,11798854,25093700,Family Adventure,2043.0,Spring,5.0,PG,94.0,All Time Domestic
4,3 Ninjas Knuckle Up,3Ninjas,1995-03-10,NaN,413479,845000,Unknown,52.0,Winter,3.0,PG-13,87.0,All Time Domestic


In [141]:
#Creating Franchise Sequence Number
Franchise_Movies['FranchiseNum'] = 1
for i in range(1, len(Franchise_Movies)):
    if Franchise_Movies.loc[i-1,'Franchise'] == Franchise_Movies.loc[i,'Franchise']:
        Franchise_Movies.loc[i, 'FranchiseNum'] = Franchise_Movies.loc[i-1, 'FranchiseNum'] + 1
Franchise_Movies.head()

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director,FranchiseNum
0,300,300,2007-03-09,6.500e+07,210614939,272146300,Period Action,3280.0,Winter,3.0,R,117.0,Zack Snyder,1
1,300: Rise of An Empire,300,2014-03-07,1.100e+08,106580051,118549400,Period Action,3490.0,Winter,3.0,R,102.0,Noam Murro,2
2,3 Ninjas,3Ninjas,1992-08-07,NaN,29000301,62123600,Family Adventure,1954.0,Summer,8.0,PG,85.0,Jon Turteltaub,1
3,3 Ninjas Kick Back,3Ninjas,1994-05-06,NaN,11798854,25093700,Family Adventure,2043.0,Spring,5.0,PG,94.0,All Time Domestic,2
4,3 Ninjas Knuckle Up,3Ninjas,1995-03-10,NaN,413479,845000,Unknown,52.0,Winter,3.0,PG-13,87.0,All Time Domestic,3


### Tackling missing values in budget  --  Tackle after you merge outter

In [142]:
len(Franchise_Movies[(Franchise_Movies['budget'].isnull())])

277

In [143]:
mask = (Franchise_Movies['budget'].isnull())
Franchise_Movies[(mask)]

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director,FranchiseNum
2,3 Ninjas,3Ninjas,1992-08-07,NaN,29000301,62123600,Family Adventure,1954.0,Summer,8.0,PG,85.0,Jon Turteltaub,1
3,3 Ninjas Kick Back,3Ninjas,1994-05-06,NaN,11798854,25093700,Family Adventure,2043.0,Spring,5.0,PG,94.0,All Time Domestic,2
4,3 Ninjas Knuckle Up,3Ninjas,1995-03-10,NaN,413479,845000,Unknown,52.0,Winter,3.0,PG-13,87.0,All Time Domestic,3
5,3 Ninjas: High Noon at Mega Mountain,3Ninjas,1998-04-10,NaN,375805,712300,Unknown,120.0,Spring,4.0,PG,93.0,Sean McNamara,4
14,Aliens,Alien,1986-07-18,NaN,85160248,204063200,Sci-Fi Horror,1454.0,Summer,7.0,R,137.0,James Cameron,2
18,Aliens Vs. Predator - Requiem,Alien,2007-12-25,NaN,41797066,52660200,Sci-Fi Horror,2617.0,Winter,12.0,R,86.0,Greg Strause,6
21,The Chipmunk Adventure,AlvinandtheChipmunks,1987-05-22,NaN,6804312,15470700,Animation,916.0,Spring,5.0,G,76.0,Randy Edelman,1
25,Alvin and the Chipmunks: The Road Chip,AlvinandtheChipmunks,2015-12-18,NaN,85886987,88191500,Family Comedy,3705.0,Winter,12.0,PG,86.0,Christina Applegate,5
26,American Ninja,AmericanNinja,1985-08-30,NaN,10499694,26293600,Action,672.0,Summer,8.0,R,95.0,All Time Domestic,1
27,American Ninja 2,AmericanNinja,1987-05-01,NaN,4000000,9094600,Action,397.0,Spring,5.0,R,89.0,All Time Domestic,2


## Budget Set1

In [144]:
with open('../Data/FinalBudget.json') as data_file2:    
    data2 = json.load(data_file2)

In [145]:
budget = pd.DataFrame(data2)
budget.head()

,Budget,Dom_Gross,Movie,Rel_Date,Wor_Gross,url
0,"[$15,000,000]","[$103,046,663]",[Scream],[12/20/1996],"[$173,046,663]",http://www.the-numbers.com/movie/budgets/all/
1,"[$15,000,000]","[$102,003,019]",[Jackass Presents: Bad Grandpa],[10/25/2013],"[$160,903,019]",http://www.the-numbers.com/movie/budgets/all/
2,"[$15,000,000]","[$65,192,350]",[Lethal Weapon],[3/6/1987],"[$120,192,350]",http://www.the-numbers.com/movie/budgets/all/
3,"[$15,000,000]","[$64,003,625]",[Diary of a Wimpy Kid],[3/19/2010],"[$76,954,311]",http://www.the-numbers.com/movie/budgets/all/
4,"[$15,000,000]","[$58,269,151]",[Halloween],[8/31/2007],"[$77,514,401]",http://www.the-numbers.com/movie/budgets/all/


In [146]:
#Extracting Movie & Rel_Date from list 
budget['MovieName'] = budget['Movie'].map(lambda x: x[0])
budget['Rel_Date'] = budget['Rel_Date'].map(lambda x: x[0])


In [147]:
#Extracting the money columns and cleaning
def StripMoneyBudget(x):
    if x != np.nan:
        a=''.join(x)
        a=a.replace(',', '')
        a=a.replace('$', '')
        return int(a)
    else:
        return None

In [148]:
budget['BUDGET'] = budget['Budget'].map(StripMoneyBudget)
budget['DOM_GROSS'] = budget['Dom_Gross'].map(StripMoneyBudget)
budget['WORLD_GROSS'] = budget['Wor_Gross'].map(StripMoneyBudget)

In [149]:
budget.head()

,Budget,Dom_Gross,Movie,Rel_Date,Wor_Gross,url,MovieName,BUDGET,DOM_GROSS,WORLD_GROSS
0,"[$15,000,000]","[$103,046,663]",[Scream],12/20/1996,"[$173,046,663]",http://www.the-numbers.com/movie/budgets/all/,Scream,15000000,103046663,173046663
1,"[$15,000,000]","[$102,003,019]",[Jackass Presents: Bad Grandpa],10/25/2013,"[$160,903,019]",http://www.the-numbers.com/movie/budgets/all/,Jackass Presents: Bad Grandpa,15000000,102003019,160903019
2,"[$15,000,000]","[$65,192,350]",[Lethal Weapon],3/6/1987,"[$120,192,350]",http://www.the-numbers.com/movie/budgets/all/,Lethal Weapon,15000000,65192350,120192350
3,"[$15,000,000]","[$64,003,625]",[Diary of a Wimpy Kid],3/19/2010,"[$76,954,311]",http://www.the-numbers.com/movie/budgets/all/,Diary of a Wimpy Kid,15000000,64003625,76954311
4,"[$15,000,000]","[$58,269,151]",[Halloween],8/31/2007,"[$77,514,401]",http://www.the-numbers.com/movie/budgets/all/,Halloween,15000000,58269151,77514401


In [150]:
#Changing Date to Datetime
budget['My_RDate'] = pd.to_datetime(budget['Rel_Date'])
budget.head()

,Budget,Dom_Gross,Movie,Rel_Date,Wor_Gross,url,MovieName,BUDGET,DOM_GROSS,WORLD_GROSS,My_RDate
0,"[$15,000,000]","[$103,046,663]",[Scream],12/20/1996,"[$173,046,663]",http://www.the-numbers.com/movie/budgets/all/,Scream,15000000,103046663,173046663,1996-12-20
1,"[$15,000,000]","[$102,003,019]",[Jackass Presents: Bad Grandpa],10/25/2013,"[$160,903,019]",http://www.the-numbers.com/movie/budgets/all/,Jackass Presents: Bad Grandpa,15000000,102003019,160903019,2013-10-25
2,"[$15,000,000]","[$65,192,350]",[Lethal Weapon],3/6/1987,"[$120,192,350]",http://www.the-numbers.com/movie/budgets/all/,Lethal Weapon,15000000,65192350,120192350,1987-03-06
3,"[$15,000,000]","[$64,003,625]",[Diary of a Wimpy Kid],3/19/2010,"[$76,954,311]",http://www.the-numbers.com/movie/budgets/all/,Diary of a Wimpy Kid,15000000,64003625,76954311,2010-03-19
4,"[$15,000,000]","[$58,269,151]",[Halloween],8/31/2007,"[$77,514,401]",http://www.the-numbers.com/movie/budgets/all/,Halloween,15000000,58269151,77514401,2007-08-31


In [151]:
ABudget= budget.filter(['MovieName','My_RDate','BUDGET','DOM_GROSS','WORLD_GROSS'])

In [152]:
ABudget   

,MovieName,My_RDate,BUDGET,DOM_GROSS,WORLD_GROSS
0,Scream,1996-12-20,15000000,103046663,173046663
1,Jackass Presents: Bad Grandpa,2013-10-25,15000000,102003019,160903019
2,Lethal Weapon,1987-03-06,15000000,65192350,120192350
3,Diary of a Wimpy Kid,2010-03-19,15000000,64003625,76954311
4,Halloween,2007-08-31,15000000,58269151,77514401
5,The Woman in Black,2012-02-03,15000000,54333290,128955898
6,Halloween: Resurrection,2002-07-12,15000000,30259652,37659652
7,Book of Shadows: Blair Witch 2,2000-10-27,15000000,26421314,47721314
8,Highlander: Endgame,2000-09-01,15000000,12801190,12801190
9,Superman IV: The Quest for Peace,1987-07-24,17000000,11227824,11227824


## Budget Set 2

In [153]:
with open('../Data/missbudget.json') as data_file3:    
    data3 = json.load(data_file3)

In [154]:
Budget2 = pd.DataFrame(data3)
Budget2.head()

,Budget,Dom_Gross,MovieBOM,MovieNumbers,Rel_Date,Wor_Gross,url
0,"[$14,000,000]","[$109,306,210]",Crocodile Dundee II,[Crocodile Dundee 2],[5/25/1988],"[$239,606,210]",http://www.the-numbers.com/movie/budgets/all/
1,"[$15,000,000]","[$65,192,350]",Lethal Weapon 2,[Lethal Weapon],[3/6/1987],"[$120,192,350]",http://www.the-numbers.com/movie/budgets/all/
2,"[$16,000,000]","[$127,175,374]",Dumb and Dumber,[Dumb & Dumber],[12/16/1994],"[$246,400,000]",http://www.the-numbers.com/movie/budgets/all/
3,"[$17,000,000]","[$55,041,738]",Halloween: H20,[Halloween: H2O],[8/5/1998],"[$55,041,738]",http://www.the-numbers.com/movie/budgets/all/
4,"[$19,000,000]","[$157,019,771]",Scary Movie 5,[Scary Movie],[7/7/2000],"[$277,200,000]",http://www.the-numbers.com/movie/budgets/all/


In [155]:
#Extracting Movie & Rel_Date from list 
Budget2['MovieNumbers'] = Budget2['MovieNumbers'].map(lambda x: x[0])
Budget2['Rel_Date'] = Budget2['Rel_Date'].map(lambda x: x[0])

In [156]:
#Extracting the money columns and cleaning
def StripMoneyBudget(x):
    if x != np.nan:
        a=''.join(x)
        a=a.replace(',', '')
        a=a.replace('$', '')
        return int(a)
    else:
        return None

In [157]:
Budget2['BUDGET'] = Budget2['Budget'].map(StripMoneyBudget)
Budget2['DOM_GROSS'] = Budget2['Dom_Gross'].map(StripMoneyBudget)
Budget2['WORLD_GROSS'] = Budget2['Wor_Gross'].map(StripMoneyBudget)

In [158]:
Budget2.head()

,Budget,Dom_Gross,MovieBOM,MovieNumbers,Rel_Date,Wor_Gross,url,BUDGET,DOM_GROSS,WORLD_GROSS
0,"[$14,000,000]","[$109,306,210]",Crocodile Dundee II,Crocodile Dundee 2,5/25/1988,"[$239,606,210]",http://www.the-numbers.com/movie/budgets/all/,14000000,109306210,239606210
1,"[$15,000,000]","[$65,192,350]",Lethal Weapon 2,Lethal Weapon,3/6/1987,"[$120,192,350]",http://www.the-numbers.com/movie/budgets/all/,15000000,65192350,120192350
2,"[$16,000,000]","[$127,175,374]",Dumb and Dumber,Dumb & Dumber,12/16/1994,"[$246,400,000]",http://www.the-numbers.com/movie/budgets/all/,16000000,127175374,246400000
3,"[$17,000,000]","[$55,041,738]",Halloween: H20,Halloween: H2O,8/5/1998,"[$55,041,738]",http://www.the-numbers.com/movie/budgets/all/,17000000,55041738,55041738
4,"[$19,000,000]","[$157,019,771]",Scary Movie 5,Scary Movie,7/7/2000,"[$277,200,000]",http://www.the-numbers.com/movie/budgets/all/,19000000,157019771,277200000


In [159]:
#Changing Date to Datetime
Budget2['My_RDate'] = pd.to_datetime(Budget2['Rel_Date'])
Budget2.head()

,Budget,Dom_Gross,MovieBOM,MovieNumbers,Rel_Date,Wor_Gross,url,BUDGET,DOM_GROSS,WORLD_GROSS,My_RDate
0,"[$14,000,000]","[$109,306,210]",Crocodile Dundee II,Crocodile Dundee 2,5/25/1988,"[$239,606,210]",http://www.the-numbers.com/movie/budgets/all/,14000000,109306210,239606210,1988-05-25
1,"[$15,000,000]","[$65,192,350]",Lethal Weapon 2,Lethal Weapon,3/6/1987,"[$120,192,350]",http://www.the-numbers.com/movie/budgets/all/,15000000,65192350,120192350,1987-03-06
2,"[$16,000,000]","[$127,175,374]",Dumb and Dumber,Dumb & Dumber,12/16/1994,"[$246,400,000]",http://www.the-numbers.com/movie/budgets/all/,16000000,127175374,246400000,1994-12-16
3,"[$17,000,000]","[$55,041,738]",Halloween: H20,Halloween: H2O,8/5/1998,"[$55,041,738]",http://www.the-numbers.com/movie/budgets/all/,17000000,55041738,55041738,1998-08-05
4,"[$19,000,000]","[$157,019,771]",Scary Movie 5,Scary Movie,7/7/2000,"[$277,200,000]",http://www.the-numbers.com/movie/budgets/all/,19000000,157019771,277200000,2000-07-07


In [160]:
BBudget= Budget2.filter(['MovieBOM','MovieNumbers','My_RDate','BUDGET','DOM_GROSS','WORLD_GROSS'])

In [161]:
BBudget .head() 

,MovieBOM,MovieNumbers,My_RDate,BUDGET,DOM_GROSS,WORLD_GROSS
0,Crocodile Dundee II,Crocodile Dundee 2,1988-05-25,14000000,109306210,239606210
1,Lethal Weapon 2,Lethal Weapon,1987-03-06,15000000,65192350,120192350
2,Dumb and Dumber,Dumb & Dumber,1994-12-16,16000000,127175374,246400000
3,Halloween: H20,Halloween: H2O,1998-08-05,17000000,55041738,55041738
4,Scary Movie 5,Scary Movie,2000-07-07,19000000,157019771,277200000


In [162]:
CBudget= BBudget.filter(['MovieBOM','My_RDate','BUDGET','DOM_GROSS','WORLD_GROSS'])

In [163]:
CBudget.rename(columns={'MovieBOM': 'MovieName'}, inplace=True)
CBudget.head()

,MovieName,My_RDate,BUDGET,DOM_GROSS,WORLD_GROSS
0,Crocodile Dundee II,1988-05-25,14000000,109306210,239606210
1,Lethal Weapon 2,1987-03-06,15000000,65192350,120192350
2,Dumb and Dumber,1994-12-16,16000000,127175374,246400000
3,Halloween: H20,1998-08-05,17000000,55041738,55041738
4,Scary Movie 5,2000-07-07,19000000,157019771,277200000


## Merging BBudget and CBudget with Franchise_Movies

In [164]:
len(Franchise_Movies), len(CBudget)

(784, 144)

In [165]:
DataB = Franchise_Movies.merge(CBudget, 
                       left_on=('MovieTitle', 'Release_Date'), 
                       right_on=('MovieName', 'My_RDate'), 
                       how="inner")
DataB.tail(25)

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director,FranchiseNum,MovieName,My_RDate,BUDGET,DOM_GROSS,WORLD_GROSS
48,The Huntsman: Winter's War,SnowWhiteandtheHuntsman,2016-04-22,1.150e+08,48390190,49282200,Action / Adventure,3802.0,Spring,4.0,PG-13,114.0,Writers:,2,The Huntsman: Winter's War,2016-04-22,115000000,48003015,165149302
49,The SpongeBob SquarePants Movie,SpongeBob,2004-11-19,3.000e+07,85417988,122068300,Animation,3307.0,Fall,11.0,PG,90.0,Alec Baldwin*,1,The SpongeBob SquarePants Movie,2004-11-19,30000000,85416609,142051255
50,Spy Kids 3D: Game Over,SpyKids,2003-07-25,3.800e+07,111761982,164762400,Family Adventure,3388.0,Summer,7.0,PG,85.0,Robert Rodriguez,3,Spy Kids 3D: Game Over,2003-07-25,40000000,111760631,167851995
51,Star Wars: Episode I - The Phantom Menace,StarWars,1999-05-19,1.150e+08,431088295,754101400,Sci-Fi Fantasy,3126.0,Spring,5.0,PG,133.0,George Lucas,6,Star Wars: Episode I - The Phantom Menace,1999-05-19,115000000,474544677,1027044677
52,Star Wars: Episode II - Attack of the Clones,StarWars,2002-05-16,1.150e+08,302191252,462389000,Sci-Fi Fantasy,3161.0,Spring,5.0,PG,132.0,George Lucas,7,Star Wars: Episode II - Attack of the Clones,2002-05-16,115000000,310676740,656695615
53,Star Wars: Episode III - Revenge of the Sith,StarWars,2005-05-19,1.130e+08,380270577,527395500,Sci-Fi Fantasy,3663.0,Spring,5.0,PG-13,146.0,George Lucas,8,Star Wars: Episode III - Revenge of the Sith,2005-05-19,115000000,380270577,848998877
54,Star Wars: The Force Awakens,StarWars,2015-12-18,2.450e+08,936662225,961143200,Sci-Fi Fantasy,4134.0,Winter,12.0,PG-13,136.0,J.J. Abrams,9,Star Wars: The Force Awakens,2015-12-18,306000000,936662225,2058662225
55,Step Up 3-D,StepUp,2010-08-06,3.000e+07,42400223,47413600,Music Drama,2439.0,Summer,8.0,PG-13,97.0,Jon Chu,3,Step Up 3-D,2010-08-06,30000000,42400223,165889117
56,Teenage Mutant Ninja Turtles (2014),TeenageMutantNinjaTurtles,2014-08-08,1.250e+08,191204754,210213000,Action / Adventure,3980.0,Summer,8.0,PG-13,101.0,Jonathan Liebesman,5,Teenage Mutant Ninja Turtles (2014),2014-08-08,125000000,191204754,485004754
57,Bad News Bears,TheBadNewsBears,2005-07-22,3.500e+07,32868349,45585000,Sports Comedy,3183.0,Summer,7.0,PG-13,111.0,Richard Linklater,2,Bad News Bears,2005-07-22,35000000,32868349,33500620


In [166]:
len(DataB)

73

In [167]:
len(Franchise_Movies), len(ABudget)

(784, 518)

In [168]:
DataA = Franchise_Movies.merge(ABudget, 
                       left_on=('MovieTitle'), 
                       right_on=('MovieName'), 
                       how="inner")
DataA.head(25)

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director,FranchiseNum,MovieName,My_RDate,BUDGET,DOM_GROSS,WORLD_GROSS
0,300,300,2007-03-09,6.500e+07,210614939,272146300,Period Action,3280.0,Winter,3.0,R,117.0,Zack Snyder,1,300,2007-03-09,60000000,210614939,454161935
1,3 Ninjas Kick Back,3Ninjas,1994-05-06,NaN,11798854,25093700,Family Adventure,2043.0,Spring,5.0,PG,94.0,All Time Domestic,2,3 Ninjas Kick Back,1994-05-06,20000000,11744960,11744960
2,A Haunted House,AHauntedHouse,2013-01-11,2.500e+06,40041683,44832600,Comedy,2160.0,Winter,1.0,R,80.0,Michael Tiddes,1,A Haunted House,2013-01-11,2500000,40041683,59922558
3,A Haunted House 2,AHauntedHouse,2014-04-18,4.000e+06,17329486,18494500,Horror Comedy,2310.0,Spring,4.0,R,87.0,Michael Tiddes,2,A Haunted House 2,2014-04-18,4000000,17329487,21206861
4,Kiss the Girls,AlexCross,1997-10-03,2.700e+07,60527873,117231500,Thriller,2406.0,Fall,10.0,R,117.0,Morgan Freeman,1,Kiss the Girls,1997-10-03,27000000,60527873,60527873
5,Along Came a Spider,AlexCross,2001-04-06,6.000e+07,74078174,116352500,Thriller,2573.0,Spring,4.0,R,103.0,Lee Tamahori,2,Along Came a Spider,2001-04-06,28000000,74058698,105159085
6,Alex Cross,AlexCross,2012-10-19,3.500e+07,25888412,28590700,Crime Thriller,2541.0,Fall,10.0,PG-13,101.0,Rob Cohen,3,Alex Cross,2012-10-19,35000000,25888412,35426759
7,Alice Through the Looking Glass,AliceinWonderland,2016-05-27,1.700e+08,77041381,78499700,Family Adventure,3763.0,Spring,5.0,PG,112.0,Writer:,2,Alice Through the Looking Glass,2016-05-27,170000000,77042381,277442381
8,Alien,Alien,NaT,1.100e+07,78944891,279609600,Sci-Fi Horror,757.0,,NaN,R,117.0,Ridley Scott,1,Alien,1979-05-25,9000000,80930630,203630630
9,Aliens,Alien,1986-07-18,NaN,85160248,204063200,Sci-Fi Horror,1454.0,Summer,7.0,R,137.0,James Cameron,2,Aliens,1986-07-18,17000000,85160248,183316455


In [169]:
len(DataA)

520

### Concatenating DataA & DataB

In [170]:
frames = [DataA, DataB]

result = pd.concat(frames)
 

In [171]:
result

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director,FranchiseNum,MovieName,My_RDate,BUDGET,DOM_GROSS,WORLD_GROSS
0,300,300,2007-03-09,6.500e+07,210614939,272146300,Period Action,3280.0,Winter,3.0,R,117.0,Zack Snyder,1,300,2007-03-09,60000000,210614939,454161935
1,3 Ninjas Kick Back,3Ninjas,1994-05-06,NaN,11798854,25093700,Family Adventure,2043.0,Spring,5.0,PG,94.0,All Time Domestic,2,3 Ninjas Kick Back,1994-05-06,20000000,11744960,11744960
2,A Haunted House,AHauntedHouse,2013-01-11,2.500e+06,40041683,44832600,Comedy,2160.0,Winter,1.0,R,80.0,Michael Tiddes,1,A Haunted House,2013-01-11,2500000,40041683,59922558
3,A Haunted House 2,AHauntedHouse,2014-04-18,4.000e+06,17329486,18494500,Horror Comedy,2310.0,Spring,4.0,R,87.0,Michael Tiddes,2,A Haunted House 2,2014-04-18,4000000,17329487,21206861
4,Kiss the Girls,AlexCross,1997-10-03,2.700e+07,60527873,117231500,Thriller,2406.0,Fall,10.0,R,117.0,Morgan Freeman,1,Kiss the Girls,1997-10-03,27000000,60527873,60527873
5,Along Came a Spider,AlexCross,2001-04-06,6.000e+07,74078174,116352500,Thriller,2573.0,Spring,4.0,R,103.0,Lee Tamahori,2,Along Came a Spider,2001-04-06,28000000,74058698,105159085
6,Alex Cross,AlexCross,2012-10-19,3.500e+07,25888412,28590700,Crime Thriller,2541.0,Fall,10.0,PG-13,101.0,Rob Cohen,3,Alex Cross,2012-10-19,35000000,25888412,35426759
7,Alice Through the Looking Glass,AliceinWonderland,2016-05-27,1.700e+08,77041381,78499700,Family Adventure,3763.0,Spring,5.0,PG,112.0,Writer:,2,Alice Through the Looking Glass,2016-05-27,170000000,77042381,277442381
8,Alien,Alien,NaT,1.100e+07,78944891,279609600,Sci-Fi Horror,757.0,,NaN,R,117.0,Ridley Scott,1,Alien,1979-05-25,9000000,80930630,203630630
9,Aliens,Alien,1986-07-18,NaN,85160248,204063200,Sci-Fi Horror,1454.0,Summer,7.0,R,137.0,James Cameron,2,Aliens,1986-07-18,17000000,85160248,183316455


### Finding movies in FranchiseDF that have a greater budget than 0 but are not in the final Conctenated resultDF 


May want to add these 51 movies later

In [172]:
b = list(result.MovieTitle)

In [173]:
mask = (Franchise_Movies['budget'] > 0)

In [174]:
a = list(Franchise_Movies[mask].MovieTitle)

In [175]:
missing = [movie for movie in a if movie not in b ]

In [176]:
len(missing)

51

In [177]:
missing

['Alice in Wonderland (2010)',
 'An Inconvenient Sequel: Truth to Power',
 'Arthur (2011)',
 'Atlas Shrugged: Part I',
 'Avatar: Special Edition',
 "Marvel's The Avengers",
 'Blade II',
 'The Care Bears Movie ',
 'Conan the Barbarian (2011)',
 'The Mummy (2017)',
 'Die Hard 2: Die Harder',
 'Cinderella (2015)',
 'The Jungle Book (2016)',
 'Evil Dead (2013)',
 'Fantastic Four (2005)',
 'Friday the 13th (2009)',
 'Fright Night (2011)',
 'Ghostbusters (2016)',
 'Godzilla (2014)',
 'Halloween 5',
 'Halloween: H20',
 'Halloween II (2009)',
 'Home Alone 3',
 'The LEGO Movie',
 "Tyler Perry's Diary of a Mad Black Woman",
 "Tyler Perry's Madea's Family Reunion",
 'Boo! A Madea Halloween',
 'MIB 3',
 "Pirates of the Caribbean: At World's End",
 'Planet of the Apes (2001)',
 'Pokemon: The Movie 2000',
 'Poltergeist (2015)',
 'Power Rangers (2017)',
 'Psycho (1998)',
 'Red 2',
 'RoboCop (2014)',
 'Rugrats in Paris: The Movie',
 'The Empire Strikes Back',
 'Star Wars',
 'Return of the Jedi',
 'Tee

### If Data Mising in original(left) DF variables add data from variables on right to left - Release Data, Budget 

In [178]:
len(result[(result['Release_Date'].isnull())])

19

In [179]:
result.Release_Date.fillna(result['My_RDate'],inplace=True) 

In [180]:
len(result[(result['Release_Date'].isnull())])

0

In [181]:
len(result[(result['budget'].isnull())])

125

In [182]:
result.budget.fillna(result['BUDGET'],inplace=True) 

In [183]:
len(result[(result['budget'].isnull())])

0

In [184]:
len(result[(result['Franchise'].isnull())])

0

In [185]:
len(result[(result['Widest Release'].isnull())])

7

In [186]:
len(result[(result['Run_Time'].isnull())])

1

In [187]:
result.describe()

,budget,Domestic Gross,Adj_Dom_Gross,Widest Release,Month,Run_Time,FranchiseNum,BUDGET,DOM_GROSS,WORLD_GROSS
count,5.930e+02,5.930e+02,5.930e+02,586.000,574.000,592.000,593.000,5.930e+02,5.930e+02,5.930e+02
mean,6.986e+07,1.302e+08,1.873e+08,2940.949,6.882,110.209,2.685,7.009e+07,1.296e+08,3.162e+08
std,6.539e+07,1.143e+08,1.598e+08,1006.199,3.084,20.412,1.864,6.723e+07,1.153e+08,3.221e+08
min,7.000e+03,7.396e+03,8.300e+03,1.000,1.000,63.000,1.000,7.000e+03,7.396e+03,7.396e+03
25%,1.900e+07,4.807e+07,6.545e+07,2388.250,5.000,94.000,1.000,1.900e+07,4.743e+07,8.528e+07
50%,4.200e+07,9.105e+07,1.493e+08,3181.500,7.000,106.000,2.000,4.200e+07,9.071e+07,2.028e+08
75%,1.100e+08,1.810e+08,2.609e+08,3702.500,10.000,123.000,3.000,1.100e+08,1.802e+08,4.350e+08
max,2.700e+08,9.367e+08,1.139e+09,4468.000,12.000,200.000,10.000,4.250e+08,9.367e+08,2.784e+09


In [188]:
result['Widest Release'].fillna(3181,inplace=True)   #fill with median 

In [189]:
len(result[(result['Widest Release'].isnull())])

0

In [190]:
result['Run_Time'].fillna(106,inplace=True)  #fill with median 

In [191]:
len(result[(result['Run_Time'].isnull())])

0

In [192]:
len(result[(result['Month'].isnull())])

19

In [193]:
result['Month'] = result['Release_Date'].dt.month

In [194]:
len(result[(result['Month'].isnull())])

0

In [195]:
#Defining Season

def Season(x):
    RelSeason =""
    if (x == 4.0 or x==5.0):
        RelSeason = "Spring"
    if (x == 6.0 or x==7.0 or x== 8.0):
        RelSeason = "Summer"
    if (x == 9.0 or x==10.0 or x== 11.0):
        RelSeason = "Fall"
    if (x == 12.0 or x==1.0 or x== 2.0 or x== 3.0 ):
        RelSeason = "Winter"
    return RelSeason 

result['Season'] = result['Month'].map(Season)
result.tail()

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director,FranchiseNum,MovieName,My_RDate,BUDGET,DOM_GROSS,WORLD_GROSS
68,Underworld: Rise of the Lycans,Underworld,2009-01-23,3.500e+07,45802315,54582100,Action Horror,2942.0,Winter,1,R,92.0,Michael Sheen,3,Underworld: Rise of the Lycans,2009-01-23,35000000,45802315,89102315
69,Underworld Awakening,Underworld,2012-01-20,7.000e+07,62321039,70757900,Action Horror,3078.0,Winter,1,R,88.0,Mars Marlind,4,Underworld Awakening,2012-01-20,70000000,62321039,160379930
70,Universal Soldier: The Return,UniversalSoldier,1999-08-20,4.500e+07,10667893,18668800,Sci-Fi Action,1639.0,Summer,8,R,83.0,Dean Devlin,2,Universal Soldier: The Return,1999-08-20,40000000,10447421,10717421
71,National Lampoon's Vacation,Vacation,1983-07-29,1.500e+07,61399552,173283200,Comedy,1253.0,Summer,7,R,99.0,Harold Ramis,1,National Lampoon's Vacation,1983-07-29,15000000,61400000,61400000
72,xXx: The Return of Xander Cage,xXxTrilogy,2017-01-20,8.500e+07,44898413,45152400,Action / Adventure,3651.0,Winter,1,PG-13,107.0,D.J. Caruso,3,xXx: The Return of Xander Cage,2017-01-20,85000000,44898413,345114933


In [196]:
mask = (result['Franchise'].isnull())

In [197]:
result[(mask)]

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director,FranchiseNum,MovieName,My_RDate,BUDGET,DOM_GROSS,WORLD_GROSS


In [198]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 593 entries, 0 to 72
Data columns (total 19 columns):
MovieTitle        593 non-null object
Franchise         593 non-null object
Release_Date      593 non-null datetime64[ns]
budget            593 non-null float64
Domestic Gross    593 non-null int64
Adj_Dom_Gross     593 non-null int64
genre             593 non-null object
Widest Release    593 non-null float64
Season            593 non-null object
Month             593 non-null int64
rating            593 non-null object
Run_Time          593 non-null float64
Director          593 non-null object
FranchiseNum      593 non-null int64
MovieName         593 non-null object
My_RDate          593 non-null datetime64[ns]
BUDGET            593 non-null int64
DOM_GROSS         593 non-null int64
WORLD_GROSS       593 non-null int64
dtypes: datetime64[ns](2), float64(3), int64(7), object(7)
memory usage: 92.7+ KB


In [199]:
result.head()

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director,FranchiseNum,MovieName,My_RDate,BUDGET,DOM_GROSS,WORLD_GROSS
0,300,300,2007-03-09,6.500e+07,210614939,272146300,Period Action,3280.0,Winter,3,R,117.0,Zack Snyder,1,300,2007-03-09,60000000,210614939,454161935
1,3 Ninjas Kick Back,3Ninjas,1994-05-06,2.000e+07,11798854,25093700,Family Adventure,2043.0,Spring,5,PG,94.0,All Time Domestic,2,3 Ninjas Kick Back,1994-05-06,20000000,11744960,11744960
2,A Haunted House,AHauntedHouse,2013-01-11,2.500e+06,40041683,44832600,Comedy,2160.0,Winter,1,R,80.0,Michael Tiddes,1,A Haunted House,2013-01-11,2500000,40041683,59922558
3,A Haunted House 2,AHauntedHouse,2014-04-18,4.000e+06,17329486,18494500,Horror Comedy,2310.0,Spring,4,R,87.0,Michael Tiddes,2,A Haunted House 2,2014-04-18,4000000,17329487,21206861
4,Kiss the Girls,AlexCross,1997-10-03,2.700e+07,60527873,117231500,Thriller,2406.0,Fall,10,R,117.0,Morgan Freeman,1,Kiss the Girls,1997-10-03,27000000,60527873,60527873


####  Making 2 Sets of Final DFs - With WorldWide 593 rows and without 644 rows

In [200]:
# Set 1 With WorldWide

In [201]:
from copy import deepcopy
Set1 = deepcopy(result)

In [202]:
Set1.head()

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director,FranchiseNum,MovieName,My_RDate,BUDGET,DOM_GROSS,WORLD_GROSS
0,300,300,2007-03-09,6.500e+07,210614939,272146300,Period Action,3280.0,Winter,3,R,117.0,Zack Snyder,1,300,2007-03-09,60000000,210614939,454161935
1,3 Ninjas Kick Back,3Ninjas,1994-05-06,2.000e+07,11798854,25093700,Family Adventure,2043.0,Spring,5,PG,94.0,All Time Domestic,2,3 Ninjas Kick Back,1994-05-06,20000000,11744960,11744960
2,A Haunted House,AHauntedHouse,2013-01-11,2.500e+06,40041683,44832600,Comedy,2160.0,Winter,1,R,80.0,Michael Tiddes,1,A Haunted House,2013-01-11,2500000,40041683,59922558
3,A Haunted House 2,AHauntedHouse,2014-04-18,4.000e+06,17329486,18494500,Horror Comedy,2310.0,Spring,4,R,87.0,Michael Tiddes,2,A Haunted House 2,2014-04-18,4000000,17329487,21206861
4,Kiss the Girls,AlexCross,1997-10-03,2.700e+07,60527873,117231500,Thriller,2406.0,Fall,10,R,117.0,Morgan Freeman,1,Kiss the Girls,1997-10-03,27000000,60527873,60527873


In [203]:
del Set1['MovieName']
del Set1['My_RDate']
del Set1['BUDGET']
del Set1['DOM_GROSS']

In [204]:
Set1.head()

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director,FranchiseNum,WORLD_GROSS
0,300,300,2007-03-09,6.500e+07,210614939,272146300,Period Action,3280.0,Winter,3,R,117.0,Zack Snyder,1,454161935
1,3 Ninjas Kick Back,3Ninjas,1994-05-06,2.000e+07,11798854,25093700,Family Adventure,2043.0,Spring,5,PG,94.0,All Time Domestic,2,11744960
2,A Haunted House,AHauntedHouse,2013-01-11,2.500e+06,40041683,44832600,Comedy,2160.0,Winter,1,R,80.0,Michael Tiddes,1,59922558
3,A Haunted House 2,AHauntedHouse,2014-04-18,4.000e+06,17329486,18494500,Horror Comedy,2310.0,Spring,4,R,87.0,Michael Tiddes,2,21206861
4,Kiss the Girls,AlexCross,1997-10-03,2.700e+07,60527873,117231500,Thriller,2406.0,Fall,10,R,117.0,Morgan Freeman,1,60527873


In [205]:
Set1.tail()

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director,FranchiseNum,WORLD_GROSS
68,Underworld: Rise of the Lycans,Underworld,2009-01-23,3.500e+07,45802315,54582100,Action Horror,2942.0,Winter,1,R,92.0,Michael Sheen,3,89102315
69,Underworld Awakening,Underworld,2012-01-20,7.000e+07,62321039,70757900,Action Horror,3078.0,Winter,1,R,88.0,Mars Marlind,4,160379930
70,Universal Soldier: The Return,UniversalSoldier,1999-08-20,4.500e+07,10667893,18668800,Sci-Fi Action,1639.0,Summer,8,R,83.0,Dean Devlin,2,10717421
71,National Lampoon's Vacation,Vacation,1983-07-29,1.500e+07,61399552,173283200,Comedy,1253.0,Summer,7,R,99.0,Harold Ramis,1,61400000
72,xXx: The Return of Xander Cage,xXxTrilogy,2017-01-20,8.500e+07,44898413,45152400,Action / Adventure,3651.0,Winter,1,PG-13,107.0,D.J. Caruso,3,345114933


## Pickling Data

In [206]:
import pickle

with open('../Data/project_data1.pkl', 'wb') as picklefile:
    pickle.dump(Set1, picklefile)

In [209]:
!ls ../Data

Analysis1_DF.pkl      GenreDF_y.pkl         lcvGenre_sk_model.pkl
FinalBudget.json      Genre_DF.pkl          lcvMain_sk_model.pkl
Finalnew_movies.json  Movies2DF.pkl         missbudget.json
GenreDF.pkl           New2_DF.pkl           project_data1.pkl
GenreDF_OLS_fit2.pkl  OLS_Main_sk_model.pkl project_data2.pkl
GenreDF_X.pkl         flist.json            rcvMain_sk_model.pkl


In [210]:
with open("../Data/project_data1.pkl", 'rb') as picklefile: 
    my_old_data = pickle.load(picklefile)

In [211]:
my_old_data

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director,FranchiseNum,WORLD_GROSS
0,300,300,2007-03-09,6.500e+07,210614939,272146300,Period Action,3280.0,Winter,3,R,117.0,Zack Snyder,1,454161935
1,3 Ninjas Kick Back,3Ninjas,1994-05-06,2.000e+07,11798854,25093700,Family Adventure,2043.0,Spring,5,PG,94.0,All Time Domestic,2,11744960
2,A Haunted House,AHauntedHouse,2013-01-11,2.500e+06,40041683,44832600,Comedy,2160.0,Winter,1,R,80.0,Michael Tiddes,1,59922558
3,A Haunted House 2,AHauntedHouse,2014-04-18,4.000e+06,17329486,18494500,Horror Comedy,2310.0,Spring,4,R,87.0,Michael Tiddes,2,21206861
4,Kiss the Girls,AlexCross,1997-10-03,2.700e+07,60527873,117231500,Thriller,2406.0,Fall,10,R,117.0,Morgan Freeman,1,60527873
5,Along Came a Spider,AlexCross,2001-04-06,6.000e+07,74078174,116352500,Thriller,2573.0,Spring,4,R,103.0,Lee Tamahori,2,105159085
6,Alex Cross,AlexCross,2012-10-19,3.500e+07,25888412,28590700,Crime Thriller,2541.0,Fall,10,PG-13,101.0,Rob Cohen,3,35426759
7,Alice Through the Looking Glass,AliceinWonderland,2016-05-27,1.700e+08,77041381,78499700,Family Adventure,3763.0,Spring,5,PG,112.0,Writer:,2,277442381
8,Alien,Alien,1979-05-25,1.100e+07,78944891,279609600,Sci-Fi Horror,757.0,Spring,5,R,117.0,Ridley Scott,1,203630630
9,Aliens,Alien,1986-07-18,1.700e+07,85160248,204063200,Sci-Fi Horror,1454.0,Summer,7,R,137.0,James Cameron,2,183316455


# Set 2 Without WorldWide

In [212]:
Set2 = deepcopy(Set1)

In [213]:
del Set2['WORLD_GROSS']

In [214]:
Set2.head()

,MovieTitle,Franchise,Release_Date,budget,Domestic Gross,Adj_Dom_Gross,genre,Widest Release,Season,Month,rating,Run_Time,Director,FranchiseNum
0,300,300,2007-03-09,6.500e+07,210614939,272146300,Period Action,3280.0,Winter,3,R,117.0,Zack Snyder,1
1,3 Ninjas Kick Back,3Ninjas,1994-05-06,2.000e+07,11798854,25093700,Family Adventure,2043.0,Spring,5,PG,94.0,All Time Domestic,2
2,A Haunted House,AHauntedHouse,2013-01-11,2.500e+06,40041683,44832600,Comedy,2160.0,Winter,1,R,80.0,Michael Tiddes,1
3,A Haunted House 2,AHauntedHouse,2014-04-18,4.000e+06,17329486,18494500,Horror Comedy,2310.0,Spring,4,R,87.0,Michael Tiddes,2
4,Kiss the Girls,AlexCross,1997-10-03,2.700e+07,60527873,117231500,Thriller,2406.0,Fall,10,R,117.0,Morgan Freeman,1


In [215]:
with open('../Data/project_data2.pkl', 'wb') as picklefile2:
    pickle.dump(Set2, picklefile2)

In [217]:
!ls ../Data

Analysis1_DF.pkl      GenreDF_y.pkl         lcvGenre_sk_model.pkl
FinalBudget.json      Genre_DF.pkl          lcvMain_sk_model.pkl
Finalnew_movies.json  Movies2DF.pkl         missbudget.json
GenreDF.pkl           New2_DF.pkl           project_data1.pkl
GenreDF_OLS_fit2.pkl  OLS_Main_sk_model.pkl project_data2.pkl
GenreDF_X.pkl         flist.json            rcvMain_sk_model.pkl


In [218]:
!ls

DataCleaningFile.ipynb              Initial_ScrapingBeautifulSoup.ipynb
GenreAnalysis.ipynb                 RegressionAnalysis.ipynb


###  Fill Null entries in a column with values from another column in the same row 
### Fil null  values in ColA  with corresoinding values form ColB

In [ ]:
test.ColA.fillna(test['ColB'],inplace=True) 

### Fill Col A with certain value if  column B has some value

In [ ]:
result.loc[result.MovieTitle == 'Never Say Never Again', 'Franchise'] = 'James Bond'

### Check how many entries are null in the column

In [ ]:
len(test[(test['budget'].isnull())])